# EfficientNetV2

In [1]:
import os
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential, Model 
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, MaxPooling1D, GlobalAveragePooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense, Input 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau 
from keras.callbacks import TensorBoard
from keras import backend as K 
from keras.optimizers import Adam, SGD, Adadelta
from keras.regularizers import l2, l1

from keras.callbacks import CSVLogger
import sys
import time
import pickle
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from keras.utils.vis_utils import plot_model
import numpy as np
import matplotlib.pyplot as plt



2023-12-13 16:57:49.810636: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-13 16:57:49.913803: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-13 16:57:50.520477: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-12-13 16:57:50.520528: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [18]:
# Loading images from Google Drive
train_data_dir = 'images/train'
validation_data_dir = 'images/test'
nb_train_samples = 67988 
nb_validation_samples = 22716
n_classes = 101
epochs = 10
batch_size = 75

In [9]:
# Checking image format: if RGB channel is coming first or last so, model will check first and then input shape will be feeded accordingly.
img_width = 299
img_height = 299

if K.image_data_format() == 'channels_first': 
    input_shape = (3, img_width, img_height) 
else: 
    input_shape = (img_width, img_height, 3) 

In [19]:
# Train & Test Data Generators with image augmentation 

train_datagen = ImageDataGenerator(
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    zoom_range=[.8, 1],
    channel_shift_range=30,
    fill_mode='reflect')

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = 11,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = 11,
    class_mode='categorical')

Found 67988 images belonging to 101 classes.
Found 22716 images belonging to 101 classes.


In [ ]:
from keras.applications.efficientnet_v2 import EfficientNetV2S
base_model = EfficientNetV2S(weights='imagenet', include_top=False, input_tensor=Input(shape=(299, 299, 3)))
x = base_model.output
x = AveragePooling2D(pool_size=(8, 8))(x)
x = Dropout(.4)(x)
x = Flatten()(x)

predictions = Dense(n_classes,
                    kernel_regularizer=l2(0.005),
                    activity_regularizer=l1(0.005), 
                    activation='softmax')(x)

model = Model(base_model.input, predictions)

In [15]:
model.compile(
    optimizer=SGD(lr=.01, momentum=.9), 
    loss='categorical_crossentropy', 
    metrics=['accuracy'])

/home/ktg3317/.local/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [21]:
from tensorflow.keras import callbacks
cp = callbacks.ModelCheckpoint(checkpoint_path, monitor='val_accuracy',save_best_only=True,verbose=1, mode='max')
csv_logger = callbacks.CSVLogger('efficientv2/InceptionV3.log')

In [22]:
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1,
                              patience=2, min_lr=0.00001)

**Model Training**

In [25]:
model.fit(train_generator,
          steps_per_epoch = nb_train_samples // batch_size,
          validation_data=validation_generator,
          validation_steps=nb_validation_samples // batch_size,
          epochs=10,
          verbose=1,
          callbacks=[csv_logger, reduce_lr]
          )

Epoch 1/10


2023-12-12 08:03:40.465138: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-12-12 08:03:51.273924: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8903
2023-12-12 08:03:58.955502: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


906/906 [==============================] - ETA: 0s - loss: 3.0451 - accuracy: 0.4354

2023-12-12 08:32:47.005086: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.94GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-12 08:32:47.005131: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.94GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


906/906 [==============================] - 2022s 2s/step - loss: 3.0451 - accuracy: 0.4354 - val_loss: 1.9191 - val_accuracy: 0.6288 - lr: 0.0100
Epoch 2/10
906/906 [==============================] - 1303s 1s/step - loss: 1.8279 - accuracy: 0.6323 - val_loss: 1.6462 - val_accuracy: 0.6648 - lr: 0.0100
Epoch 3/10
906/906 [==============================] - 1302s 1s/step - loss: 1.5460 - accuracy: 0.6834 - val_loss: 1.5528 - val_accuracy: 0.6839 - lr: 0.0100
Epoch 4/10
906/906 [==============================] - 1305s 1s/step - loss: 1.3687 - accuracy: 0.7207 - val_loss: 1.4811 - val_accuracy: 0.7008 - lr: 0.0100
Epoch 5/10
906/906 [==============================] - 1274s 1s/step - loss: 1.2320 - accuracy: 0.7510 - val_loss: 1.4846 - val_accuracy: 0.7035 - lr: 0.0100
Epoch 6/10
906/906 [==============================] - 1273s 1s/step - loss: 1.1133 - accuracy: 0.7758 - val_loss: 1.4553 - val_accuracy: 0.7156 - lr: 0.0100
Epoch 7/10
906/906 [==============================] - 1242s 1s/step -